# Fine tuning with transformers 

* Original source: [Fine-tuning pretrained NLP models with Huggingface’s Trainer by Vincent Tan](https://towardsdatascience.com/fine-tuning-pretrained-nlp-models-with-huggingfaces-trainer-6326a4456e7b) 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /Users/bsantanna/miniforge3/envs/ml/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so
'NoneType' object has no attribute 'cadam32bit_grad_fp32'
CUDA SETUP: Loading binary /Users/bsantanna/miniforge3/envs/ml/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so...
dlopen(/Users/bsantanna/miniforge3/envs/ml/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so, 0x0006): tried: '/Users/bsantanna/miniforge3/envs/ml/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/Users/bsantanna/miniforge3/envs/ml/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cpu.so' (no such file), '/Users/bsantanna/miniforge

/Users/bsantanna/miniforge3/envs/ml/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [3]:
# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [4]:
# Read data
data = pd.read_csv("imdb-review/train_data.csv")
data = data.drop("ID",axis=1)

In [5]:
data.head()

,SentimentText,Sentiment
0,first think another disney movie might good it...,1
1,put aside dr house repeat missed desperate hou...,0
2,big fan stephen king s work film made even gre...,1
3,watched horrid thing tv needless say one movie...,0
4,truly enjoyed film acting terrific plot jeff c...,1


In [6]:
# Define pretrained tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Preprocess data
X = list(data["SentimentText"])
y = list(data["Sentiment"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [8]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [10]:
# Train pre-trained model
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=2500, training_loss=0.3498365783691406, metrics={'train_runtime': 2074.4219, 'train_samples_per_second': 27.767, 'train_steps_per_second': 3.471, 'total_flos': 5262221107200000.0, 'train_loss': 0.3498365783691406, 'epoch': 1.04})

In [13]:
# ----- 3. Predict -----#
# Load test data
test_data = pd.read_csv("imdb-review/test_data.csv")
X_test = list(test_data["Sentiment"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
test_dataset = Dataset(X_test_tokenized)

# Load trained model
model_path = "output/checkpoint-2500"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

# Define test trainer
test_trainer = Trainer(model)

# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

In [15]:
type(y_pred)

numpy.ndarray

In [16]:
test_data["Sentiment"][0]

'wow snoozer definately one bacon s worst films bad acting coupled formulatic incredulous script make yearn time wasted viewing cable television back really much say it basketball scout gets attached person he s recruiting happens belong tribe happens verge war happens decided spoiler basketball game grade f'

In [17]:
y_pred[0]

0

In [23]:
test_data["Sentiment"][1]

'white balloon crimson gold two films jafar panahi i ve seen director mines surprising amounts depth subjects seem surface slight offside panahi s seriocomic tribute iranian women standing rights do not think he s successful he s saying is not important course and it s bad that like pretty much every panahi film iranians can not see it but while film feels tad flat feels long even minutes saying did not like it though actors fantastic celebration end film infectious it s important work opinion'

In [24]:
y_pred[1]

1